In [6]:
# !pip install -U textblob

In [4]:
# For EDA now, will add more libraries as we progress
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from textblob import TextBlob

# Set nice style for plots
sns.set_theme(style='darkgrid')
sns.dark_palette("#69d", reverse=True, as_cmap=True)
sns.set_context("paper")



# 1. Read Data

Our data was taken from this database of [Google Local Data 2021](https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/googlelocal/#subsets). We previously used a smaller dataset of Google Restaurant reviews available [here](https://www.kaggle.com/datasets/denizbilginn/google-maps-restaurant-reviews), with only 1100 reviews, but as per our TF's recommendation, we will be using this larger dataset with over 10m reviews for Massachusetts alone. To speed up our EDA, we will be using a sample of 100,000 reviews.

<div style="background-color:#3F7FBF; color:white; padding:10px"> 

Our main dataset, stored in the `df` variable, contains 100,000 reviews. Our columns of interest are `rating`, which is the rating given by the user, `text`, which is the review text, and `gmap_id`, which is the Google Maps ID of the business. 

We also have a metadata dataset, stored in the `df_meta` variable, which contains information about each business in Massachusetts. The columns of interest are `gmap_id`, allowing us to join this dataset with the main one, `name`, which is the name of the business, and `description`, which is a brief description of the business. There is also a variable `category` that organizes the businesses into sectors such as non-profits, gyms, restaurants, etc. 

In [5]:
# Load the data
with open('/Users/driesrooryck/Desktop/s24/ac209b/project/restaurant-rec-system/data/cleaned_data.csv', 'r') as f:
    data = f.readlines()

# Convert to dataframe
data_json_str = "[" + ','.join(data) + "]"
df = pd.read_csv(StringIO(data_json_str))

/var/folders/zx/klnklk4d1vq5zns_hyg6xc0h0000gn/T/ipykernel_19494/3734935933.py:7: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(StringIO(data_json_str))


In [7]:
# Examine df
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
Index: 1559244 entries, nan to ]
Data columns (total 16 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   [               1559243 non-null  float64
 1   user_id         1559243 non-null  float64
 2   rating          1559243 non-null  float64
 3   text            1559243 non-null  object 
 4   gmap_id         1559243 non-null  object 
 5   name_y          1559243 non-null  object 
 6   latitude        1559243 non-null  float64
 7   longitude       1559243 non-null  float64
 8   category        1559243 non-null  object 
 9   avg_rating      1559243 non-null  float64
 10  num_of_reviews  1559243 non-null  float64
 11  price           1559243 non-null  float64
 12  Dining options  1559243 non-null  object 
 13  Popular for     1559243 non-null  object 
 14  Atmosphere      1559243 non-null  object 
 15  Crowd           1559243 non-null  object 
dtypes: float64(8), object(8)
memory usage: 202.2+

In [8]:
# Also load the whole meta-Massachusetts.json file. This contains metadata about the businesses, 
# since the reviews only contain the business id.
with open('data/meta-Massachusetts.json', 'r') as f:
    data_meta = f.readlines()

data_meta_str = "[" + ','.join(data_meta) + "]"
df_meta = pd.read_json(StringIO(data_meta_str))

# Filter dataframe so that it only contains restaurants
df_meta = df_meta[df_meta['category'].apply(lambda x: isinstance(x, list) and any('restaurant' in category.lower() for category in x) if x is not None else False)]

In [9]:
# Examine df
print(df_meta.info())
print(df_meta.head())

<class 'pandas.core.frame.DataFrame'>
Index: 16079 entries, 14 to 92514
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              16079 non-null  object 
 1   address           16055 non-null  object 
 2   gmap_id           16079 non-null  object 
 3   description       9341 non-null   object 
 4   latitude          16079 non-null  float64
 5   longitude         16079 non-null  float64
 6   category          16079 non-null  object 
 7   avg_rating        16079 non-null  float64
 8   num_of_reviews    16079 non-null  int64  
 9   price             11987 non-null  object 
 10  hours             15127 non-null  object 
 11  MISC              15994 non-null  object 
 12  state             11728 non-null  object 
 13  relative_results  15090 non-null  object 
 14  url               16079 non-null  object 
dtypes: float64(3), int64(1), object(11)
memory usage: 2.0+ MB
None
                            

In [10]:
# Merging the dataframes on 'gmap_id'
# 'inner' will only include rows that have matching 'gmap_id' in both dataframes
df_combined = pd.merge(df, df_meta, on='gmap_id', how='inner')

In [11]:
# Examine the combined dataframe
print(df_combined.info())
print(df_combined.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1559281 entries, 0 to 1559280
Data columns (total 30 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   [                 1559281 non-null  float64
 1   user_id           1559281 non-null  float64
 2   rating            1559281 non-null  float64
 3   text              1559281 non-null  object 
 4   gmap_id           1559281 non-null  object 
 5   name_y            1559281 non-null  object 
 6   latitude_x        1559281 non-null  float64
 7   longitude_x       1559281 non-null  float64
 8   category_x        1559281 non-null  object 
 9   avg_rating_x      1559281 non-null  float64
 10  num_of_reviews_x  1559281 non-null  float64
 11  price_x           1559281 non-null  float64
 12  Dining options    1559281 non-null  object 
 13  Popular for       1559281 non-null  object 
 14  Atmosphere        1559281 non-null  object 
 15  Crowd             1559281 non-null  object 
 16  

<div style="background-color:#3F7FBF; color:white; padding:10px"> 

We merged the main dataset and our meta dataset based on `gmap_id` and filtered out the ones of restaurants.

We will clean up the dataframe for our purpose. From below we see that we can probably get rid of these columns: `name_x` (since we don't care about the name of the reviewer, having `user_id` is enough to identify them; `time` (for now we don't care about when the reviewers wrote the reviews); `pics` (we don't care about pictures first, but might use it in the future if time permits); `resp` (most of them are `None`); `gmap_id`(we can identify restaurants based on their names which is easier to understand); `description` (most of them are `None`); `relative_results`; `url`)

We see that there are duplicates in our merged dataset. We will drop the duplicates after we clean up the dataframe to only keep necessary columns.

In [12]:
df_combined.head()

,[,user_id,rating,text,gmap_id,name_y,latitude_x,longitude_x,category_x,avg_rating_x,...,longitude_y,category_y,avg_rating_y,num_of_reviews_y,price_y,hours,MISC,state,relative_results,url
0,24.0,1.085140e+20,5.0,Amazing food and T is an amazing man.,0x89e3169821e62d4d:0x14ff0683c1ebca0e,Three Star Pizza,42.559072,-70.881542,"['Pizza restaurant', 'Italian restaurant', 'De...",3.9,...,-70.881542,"[Pizza restaurant, Italian restaurant, Deliver...",3.9,48,$$,"[[Thursday, 11AM–8PM], [Friday, 11AM–10PM], [S...","{'Service options': ['Takeout', 'Delivery'], '...",Permanently closed,"[0x89e31418f27b6a29:0x2fd2e82a6f96214c, 0x89e3...",https://www.google.com/maps/place//data=!4m2!3...
1,24.0,1.085140e+20,5.0,Amazing food and T is an amazing man.,0x89e3169821e62d4d:0x14ff0683c1ebca0e,Three Star Pizza,42.559072,-70.881542,"['Pizza restaurant', 'Italian restaurant', 'De...",3.9,...,-70.881542,"[Pizza restaurant, Italian restaurant, Deliver...",3.9,48,$$,"[[Thursday, 11AM–8PM], [Friday, 11AM–10PM], [S...","{'Service options': ['Takeout', 'Delivery'], '...",Permanently closed,"[0x89e31418f27b6a29:0x2fd2e82a6f96214c, 0x89e3...",https://www.google.com/maps/place//data=!4m2!3...
2,25.0,1.002992e+20,3.0,I didn't actually try it but just felt like gi...,0x89e3169821e62d4d:0x14ff0683c1ebca0e,Three Star Pizza,42.559072,-70.881542,"['Pizza restaurant', 'Italian restaurant', 'De...",3.9,...,-70.881542,"[Pizza restaurant, Italian restaurant, Deliver...",3.9,48,$$,"[[Thursday, 11AM–8PM], [Friday, 11AM–10PM], [S...","{'Service options': ['Takeout', 'Delivery'], '...",Permanently closed,"[0x89e31418f27b6a29:0x2fd2e82a6f96214c, 0x89e3...",https://www.google.com/maps/place//data=!4m2!3...
3,25.0,1.002992e+20,3.0,I didn't actually try it but just felt like gi...,0x89e3169821e62d4d:0x14ff0683c1ebca0e,Three Star Pizza,42.559072,-70.881542,"['Pizza restaurant', 'Italian restaurant', 'De...",3.9,...,-70.881542,"[Pizza restaurant, Italian restaurant, Deliver...",3.9,48,$$,"[[Thursday, 11AM–8PM], [Friday, 11AM–10PM], [S...","{'Service options': ['Takeout', 'Delivery'], '...",Permanently closed,"[0x89e31418f27b6a29:0x2fd2e82a6f96214c, 0x89e3...",https://www.google.com/maps/place//data=!4m2!3...
4,26.0,1.176323e+20,5.0,I am a huge fan of their sandwiches! Made to o...,0x89e3169821e62d4d:0x14ff0683c1ebca0e,Three Star Pizza,42.559072,-70.881542,"['Pizza restaurant', 'Italian restaurant', 'De...",3.9,...,-70.881542,"[Pizza restaurant, Italian restaurant, Deliver...",3.9,48,$$,"[[Thursday, 11AM–8PM], [Friday, 11AM–10PM], [S...","{'Service options': ['Takeout', 'Delivery'], '...",Permanently closed,"[0x89e31418f27b6a29:0x2fd2e82a6f96214c, 0x89e3...",https://www.google.com/maps/place//data=!4m2!3...


In [13]:
columns_to_drop = ['name_x', 'time', 'pics', 'resp', 'gmap_id', 'description', 'relative_results', 'url']
df_cleaned = df_combined.drop(columns=columns_to_drop)

# Drop duplicates based on `user_id` and `gmap_id` to ensure unique reviews per user per restaurant
df_cleaned = df_cleaned.drop_duplicates(subset=['user_id', 'gmap_id'])

df_cleaned.head()

KeyError: "['name_x', 'time', 'pics', 'resp'] not found in axis"

In [ ]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4135196 entries, 0 to 4182566
Data columns (total 15 columns):
 #   Column          Dtype  
---  ------          -----  
 0   user_id         float64
 1   rating          float64
 2   text            object 
 3   gmap_id         object 
 4   name_y          object 
 5   address         object 
 6   latitude        float64
 7   longitude       float64
 8   category        object 
 9   avg_rating      float64
 10  num_of_reviews  int64  
 11  price           object 
 12  hours           object 
 13  MISC            object 
 14  state           object 
dtypes: float64(5), int64(1), object(9)
memory usage: 504.8+ MB


<div style="background-color:#3F7FBF; color:white; padding:10px"> 

The `MISC` column contains dictionaries with useful information. We decided to further process this column and extract the attributes out as new columns of our dataframe.

In [ ]:
print(df_cleaned.loc[0, 'MISC'])

{'Service options': ['Takeout', 'Delivery'], 'Accessibility': ['Wheelchair accessible entrance'], 'Offerings': ['Comfort food', 'Late-night food', 'Vegetarian options'], 'Dining options': ['Lunch', 'Dinner', 'Dessert'], 'Payments': ['Debit cards']}


In [ ]:
# First, ensure all entries in 'MISC' are dictionaries; replace None with empty dictionaries
df_cleaned['MISC'] = df_cleaned['MISC'].apply(lambda x: x if isinstance(x, dict) else {})

# Convert the 'MISC' column to a DataFrame where each key in the dictionary becomes a column
misc_expanded = pd.DataFrame(df_cleaned['MISC'].tolist())

In [ ]:
# Join the expanded 'MISC' DataFrame with the original 'df_cleaned', excluding the 'MISC' column
df_expanded = pd.concat([df_cleaned.drop(columns=['MISC']), misc_expanded], axis=1)

In [ ]:
df_expanded.head()

,user_id,rating,text,gmap_id,name_y,address,latitude,longitude,category,avg_rating,...,Payments,Highlights,Popular for,Amenities,Atmosphere,Health & safety,Crowd,Planning,From the business,Health and safety
0,1.053246e+20,4.0,What a great experience. I tried the chicken t...,0x89e3169821e62d4d:0x14ff0683c1ebca0e,Three Star Pizza,"Three Star Pizza, 409 Cabot St #1, Beverly, MA...",42.559072,-70.881542,"['Pizza restaurant', 'Italian restaurant', 'De...",3.9,...,['Debit cards'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.115933e+20,5.0,I've probably driven past this place a million...,0x89e3169821e62d4d:0x14ff0683c1ebca0e,Three Star Pizza,"Three Star Pizza, 409 Cabot St #1, Beverly, MA...",42.559072,-70.881542,"['Pizza restaurant', 'Italian restaurant', 'De...",3.9,...,['Debit cards'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.045125e+20,5.0,The food was fantastic. The staff was very fr...,0x89e3169821e62d4d:0x14ff0683c1ebca0e,Three Star Pizza,"Three Star Pizza, 409 Cabot St #1, Beverly, MA...",42.559072,-70.881542,"['Pizza restaurant', 'Italian restaurant', 'De...",3.9,...,['Debit cards'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.085975e+20,5.0,"The owner, T, is a wonderful man. Super friend...",0x89e3169821e62d4d:0x14ff0683c1ebca0e,Three Star Pizza,"Three Star Pizza, 409 Cabot St #1, Beverly, MA...",42.559072,-70.881542,"['Pizza restaurant', 'Italian restaurant', 'De...",3.9,...,['Debit cards'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.182263e+20,5.0,Just stopped in for a couple of grilled chicke...,0x89e3169821e62d4d:0x14ff0683c1ebca0e,Three Star Pizza,"Three Star Pizza, 409 Cabot St #1, Beverly, MA...",42.559072,-70.881542,"['Pizza restaurant', 'Italian restaurant', 'De...",3.9,...,['Debit cards'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_expanded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4182567 entries, 0 to 3957456
Data columns (total 28 columns):
 #   Column             Dtype  
---  ------             -----  
 0   user_id            float64
 1   rating             float64
 2   text               object 
 3   gmap_id            object 
 4   name_y             object 
 5   address            object 
 6   latitude           float64
 7   longitude          float64
 8   category           object 
 9   avg_rating         float64
 10  num_of_reviews     float64
 11  price              object 
 12  hours              object 
 13  state              object 
 14  Service options    object 
 15  Accessibility      object 
 16  Offerings          object 
 17  Dining options     object 
 18  Payments           object 
 19  Highlights         object 
 20  Popular for        object 
 21  Amenities          object 
 22  Atmosphere         object 
 23  Health & safety    object 
 24  Crowd              object 
 25  Planning           obje

In [ ]:
columns_of_interest = [
    "Service options", "Accessibility", "Offerings", "Dining options",
    "Payments", "Highlights", "Popular for", "Amenities",
    "Atmosphere", "Health & safety", "Crowd", "Planning",
    "From the business", "Health and safety"
]

# Initialize a dictionary to hold the unique values for each column
unique_values = {}

# Iterate over each column of interest
for column in columns_of_interest:
    # Extract the column's data
    column_data = df_expanded[column]
    
    # Since the data might contain lists, we need to flatten these into a single list before finding unique values
    # We'll use a set to automatically keep only unique items
    flattened_set = set()
    for item in column_data.dropna():  # Drop NA values to avoid errors
        if isinstance(item, list):  # Check if the item is a list
            flattened_set.update(item)  # Add all items in the list to the set
        else:
            flattened_set.add(item)  # Add the item itself if it's not a list
    
    # Store the unique values for this column in our dictionary
    unique_values[column] = list(flattened_set)  # Convert the set back to a list for readability

# Or, to print the unique values for all columns of interest:
for column, values in unique_values.items():
    print(f"{column}: {values}\n")

Service options: ['Same-day delivery', 'Drive-through', 'Takeaway', 'No-contact delivery', 'In-store pick-up', 'In-store pickup', 'In-store shopping', 'Takeout', 'Curbside pickup', 'Online appointments', 'Dine-in', 'Delivery', 'Outdoor seating']

Accessibility: ['Wheelchair-accessible seating', 'Wheelchair-accessible lift', 'Wheelchair accessible seating', 'Wheelchair-accessible toilet', 'Wheelchair rental', 'Wheelchair accessible restroom', 'Wheelchair accessible elevator', 'Wheelchair-accessible entrance', 'Assisted listening devices', 'Wheelchair accessible parking lot', 'Wheelchair-accessible car park', 'Wheelchair accessible entrance']

Offerings: ['Car wash', 'Organic products', 'Service guarantee', "Kids' menu", 'All you can eat', 'Cocktails', 'Late-night food', 'Healthy options', 'Braille menu', 'Small plates', 'Salad bar', 'Ethanol-free gas', 'Wine', 'Food', 'Alcohol', 'Organic dishes', 'Happy hour drinks', 'Prepared foods', 'Hard liquor', 'Happy hour food', 'Quick bite', 'Dan

In [ ]:
# Counting NaN values in each column of df_expanded
nan_counts = df_expanded.isna().sum()

# Displaying the count of NaN values per column
print(nan_counts)

user_id                47393
rating                 47393
text                 1939755
gmap_id                47371
name_y                 47371
address                48769
latitude               47371
longitude              47371
category               47371
avg_rating             47371
num_of_reviews         47371
price                 432281
hours                 156138
state                1891583
Service options       148804
Accessibility         412830
Offerings             359532
Dining options        688883
Payments             1605942
Highlights           1359587
Popular for           710651
Amenities             444311
Atmosphere            629354
Health & safety      1675107
Crowd                 666088
Planning             2167329
From the business    4008956
Health and safety    4115234
dtype: int64


<div style="background-color:#3F7FBF; color:white; padding:10px"> 
Since we will focus on reviews, we decided to drop rows with `text` (which is the review column) empty. We will also drop the rows with `rating` empty.

In [ ]:
# Drop rows where the 'text' column is NaN
df_expanded_clean = df_expanded.dropna(subset=['text', 'rating'])
df_expanded_clean.isna().sum()

user_id                    0
rating                     0
text                       0
gmap_id                    0
name_y                     0
address                  876
latitude                   0
longitude                  0
category                   0
avg_rating                 0
num_of_reviews             0
price                 228737
hours                  60330
state                 972136
Service options        79735
Accessibility         225756
Offerings             193528
Dining options        373764
Payments              865827
Highlights            733854
Popular for           381433
Amenities             239348
Atmosphere            338579
Health & safety       901788
Crowd                 361848
Planning             1165274
From the business    2150714
Health and safety    2207328
dtype: int64

<div style="background-color:#3F7FBF; color:white; padding:10px"> 
    
Let's further inspect these columns and decide either to drop them or process them.

`Planning`, `From the business`, and `Health and safety` simply just have too many missing values, so we will drop them.

We decided to drop `address` column since we will be able to infer the address based on names of the restaurants and the latitude and longitude.

`state` describes the current states of the restraurants when the dataset authors scraped it. This is not that helpful for our analysis so we will drop this column too.

There are some other variables which are not important for our analysis for now. We will drop them as well and start thinking about how to process the rest (e.g. one-hot encode).

In [ ]:
df_expanded_clean['state'].unique()

array(['Permanently closed', nan, 'Closes soon ⋅ 8:30PM ⋅ Opens 11AM Thu',
       'Closed ⋅ Opens 7AM Thu', 'Open ⋅ Closes 10PM',
       'Closes soon ⋅ 4PM ⋅ Opens 8AM Thu', 'Open ⋅ Closes 8PM',
       'Open ⋅ Closes 5PM', 'Closed ⋅ Opens 11AM',
       'Closed ⋅ Opens 11:30AM', 'Closed ⋅ Opens 5PM Thu',
       'Open ⋅ Closes 2AM', 'Closed ⋅ Opens 11AM Wed',
       'Closed ⋅ Opens 8AM Wed', 'Closed ⋅ Opens 8:30AM Wed',
       'Closed ⋅ Opens 6AM Wed', 'Closed ⋅ Opens 7:30AM Wed',
       'Open ⋅ Closes 8AM Wed', 'Opens soon ⋅ 9AM',
       'Closed ⋅ Opens 8:30AM', 'Closed ⋅ Opens 4PM Thu',
       'Closed ⋅ Opens 5AM', 'Closed ⋅ Opens 7:15AM',
       'Closes soon ⋅ 9PM ⋅ Opens 10AM Tue', 'Open 24 hours',
       'Open ⋅ Closes 9PM', 'Closed ⋅ Opens 4PM', 'Closed ⋅ Opens 7AM',
       'Closed ⋅ Opens 7:30AM Mon', 'Closed ⋅ Opens 5:30AM Mon',
       'Closed ⋅ Opens 3PM Tue', 'Closed ⋅ Opens 8:30AM Mon',
       'Closed ⋅ Opens 11AM Mon', 'Closed ⋅ Opens 12PM',
       'Closed ⋅ Opens 9AM', 'Clos

In [ ]:
df_expanded_clean['price'].unique()

array(['$$', nan, '$', '$$$', '$$$$', '₩₩', '₩', '₩₩₩'], dtype=object)

In [ ]:
df_expanded_clean['hours'].unique()

array(["[['Thursday', '11AM–8PM'], ['Friday', '11AM–10PM'], ['Saturday', '11AM–10PM'], ['Sunday', '4–10PM'], ['Monday', 'Closed'], ['Tuesday', '11AM–8PM'], ['Wednesday', '11AM–8PM']]",
       nan,
       "[['Wednesday', '11AM–8:30PM'], ['Thursday', '11AM–8:30PM'], ['Friday', '11AM–9PM'], ['Saturday', '11AM–9PM'], ['Sunday', '11AM–8:30AM'], ['Monday', '11AM–8:30AM'], ['Tuesday', '11AM–8:30PM']]",
       ...,
       "[['Friday', '11:30AM–8PM'], ['Saturday', '4–8PM'], ['Sunday', '4–8PM'], ['Monday', '11:30AM–8PM'], ['Tuesday', '11:30AM–8PM'], ['Wednesday', '11:30AM–8PM'], ['Thursday', '11:30AM–8PM']]",
       "[['Friday', '7AM–8PM'], ['Saturday', '7AM–8PM'], ['Sunday', '11AM–8PM'], ['Monday', '7AM–8PM'], ['Tuesday', '7AM–8PM'], ['Wednesday', '7AM–8PM'], ['Thursday', '7AM–8PM']]",
       "[['Thursday', 'Closed'], ['Friday', 'Closed'], ['Saturday', 'Closed'], ['Sunday', '3:35–3:36AM'], ['Monday', 'Closed'], ['Tuesday', 'Closed'], ['Wednesday', 'Closed']]"],
      dtype=object)

In [ ]:
df_expanded_clean['Service options'].unique()

array(["['Takeout', 'Delivery']", "['Delivery']",
       "['No-contact delivery', 'Delivery', 'Takeout', 'Dine-in']",
       "['Delivery', 'Takeout', 'Dine-in']", "['Takeout', 'Dine-in']",
       "['In-store shopping', 'Takeout', 'Dine-in', 'Delivery']",
       "['Takeout', 'Dine-in', 'Delivery']",
       "['Drive-through', 'Takeout', 'Dine-in', 'Delivery']",
       "['Curbside pickup', 'In-store pickup', 'In-store shopping', 'Takeout', 'Dine-in', 'Delivery']",
       "['Outdoor seating', 'Delivery']", "['Delivery', 'Takeout']",
       "['Curbside pickup', 'Takeout', 'Dine-in', 'Delivery']",
       "['In-store shopping']",
       "['Outdoor seating', 'Delivery', 'Takeout', 'Dine-in']",
       "['Dine-in', 'Delivery']",
       "['Curbside pickup', 'No-contact delivery', 'Delivery', 'Takeout', 'Dine-in']",
       nan, "['In-store shopping', 'Delivery']",
       "['Outdoor seating', 'Curbside pickup', 'No-contact delivery', 'Delivery', 'Takeout', 'Dine-in']",
       "['Curbside pickup', '

In [ ]:
df_expanded_clean['Accessibility'].unique()

array(["['Wheelchair accessible entrance']",
       "['Wheelchair accessible entrance', 'Wheelchair accessible restroom']",
       nan,
       "['Wheelchair accessible entrance', 'Wheelchair accessible parking lot', 'Wheelchair accessible seating']",
       "['Wheelchair accessible entrance', 'Wheelchair accessible seating']",
       "['Wheelchair accessible entrance', 'Wheelchair accessible parking lot', 'Wheelchair accessible restroom', 'Wheelchair accessible seating']",
       "['Wheelchair accessible entrance', 'Wheelchair accessible parking lot', 'Wheelchair accessible restroom']",
       "['Wheelchair accessible seating']",
       "['Wheelchair accessible elevator', 'Wheelchair accessible entrance', 'Wheelchair accessible parking lot', 'Wheelchair accessible restroom', 'Wheelchair accessible seating']",
       "['Wheelchair accessible restroom', 'Wheelchair accessible entrance']",
       "['Wheelchair accessible parking lot']",
       "['Wheelchair accessible parking lot', 'Wheel

In [ ]:
df_expanded_clean['Offerings'].unique()

array(["['Comfort food', 'Late-night food', 'Vegetarian options']",
       "['Coffee', 'Comfort food', 'Healthy options', 'Organic dishes', 'Quick bite', 'Vegetarian options']",
       "['Coffee', 'Comfort food', 'Healthy options', 'Quick bite', 'Small plates', 'Vegetarian options']",
       ...,
       '[\'Alcohol\', \'All you can eat\', \'Beer\', \'Braille menu\', \'Cocktails\', \'Coffee\', \'Comfort food\', \'Happy hour drinks\', \'Happy hour food\', \'Hard liquor\', "Kids\' menu", \'Late-night food\', \'Organic dishes\', \'Small plates\', \'Vegetarian options\', \'Wine\']',
       '[\'Alcohol\', \'Coffee\', \'Comfort food\', \'Food\', \'Food at bar\', \'Happy hour food\', "Kids\' menu", \'Late-night food\', \'Quick bite\']',
       '[\'Coffee\', \'Halal food\', \'Healthy options\', "Kids\' menu", \'Late-night food\', \'Quick bite\', \'Vegetarian options\']'],
      dtype=object)

In [ ]:
df_expanded_clean['Dining options'].unique()

array(["['Lunch', 'Dinner', 'Dessert']",
       "['Breakfast', 'Lunch', 'Dinner', 'Dessert']", "['Dessert']", nan,
       "['Breakfast', 'Lunch', 'Catering', 'Dessert', 'Seating']",
       "['Lunch', 'Dinner', 'Catering', 'Dessert', 'Seating']",
       "['Breakfast', 'Lunch', 'Dinner', 'Dessert', 'Seating']",
       "['Breakfast']", "['Breakfast', 'Lunch', 'Dessert']",
       "['Breakfast', 'Dessert']",
       "['Lunch', 'Dinner', 'Catering', 'Dessert']",
       "['Breakfast', 'Lunch', 'Dinner']", "['Seating']",
       "['Lunch', 'Dinner', 'Seating']",
       "['Breakfast', 'Lunch', 'Dinner', 'Catering', 'Dessert', 'Seating']",
       "['Lunch', 'Dinner']", "['Breakfast', 'Lunch']",
       "['Breakfast', 'Lunch', 'Dinner', 'Counter service', 'Dessert']",
       "['Lunch', 'Dinner', 'Catering']", "['Lunch', 'Catering']",
       "['Lunch', 'Dinner', 'Dessert', 'Seating']",
       "['Lunch', 'Catering', 'Dessert']",
       "['Lunch', 'Dinner', 'Catering', 'Seating']", "['Dinner']",
      

In [ ]:
df_expanded_clean['Payments'].unique()

array(["['Debit cards']", nan, "['NFC mobile payments']",
       "['Debit cards', 'Credit cards']",
       "['Debit cards', 'NFC mobile payments', 'Credit cards']",
       "['Debit cards', 'NFC mobile payments']",
       "['Checks', 'Debit cards', 'Credit cards']", "['Credit cards']",
       "['Cash-only']", "['Cash-only', 'Debit cards']",
       "['Cash-only', 'Credit cards']",
       "['Cash-only', 'Debit cards', 'Credit cards']",
       "['Cash-only', 'Checks', 'Debit cards', 'NFC mobile payments']",
       "['Cash-only', 'Checks', 'Debit cards', 'Credit cards']",
       "['NFC mobile payments', 'Credit cards']",
       "['Checks', 'Debit cards']",
       "['Checks', 'Debit cards', 'NFC mobile payments']",
       "['Cash-only', 'NFC mobile payments']", "['Checks']",
       "['Cash-only', 'Debit cards', 'NFC mobile payments']",
       "['Checks', 'Debit cards', 'NFC mobile payments', 'Credit cards']",
       "['Cash-only', 'Debit cards', 'NFC mobile payments', 'Credit cards']",
     

In [ ]:
df_expanded_clean['Highlights'].unique()

array([nan, "['Great coffee', 'Great dessert', 'Great tea selection']",
       "['Great coffee']", "['Fast service']",
       "['Great beer selection', 'Great cocktails']",
       "['Fast service', 'Great coffee', 'Great tea selection']",
       "['Great coffee', 'Great dessert']", "['Great dessert']",
       "['LGBTQ friendly', 'Transgender safespace']",
       "['Fast service', 'Great coffee']", "['LGBTQ friendly']",
       "['Fireplace', 'Great coffee']",
       "['Great beer selection', 'Great cocktails', 'LGBTQ friendly']",
       "['Great coffee', 'Great wine list']", "['Great cocktails']",
       "['Fireplace', 'Live music']",
       "['Great beer selection', 'Great cocktails', 'Great wine list']",
       "['Live music']", "['Great dessert', 'LGBTQ friendly']",
       "['Great cocktails', 'Live music']",
       "['Fast service', 'Great beer selection', 'Great coffee', 'Great dessert', 'Great wine list']",
       "['Great cocktails', 'Great wine list']",
       "['Great beer sele

In [ ]:
df_expanded_clean['Popular for'].unique()

array([nan, "['Breakfast', 'Lunch', 'Dinner', 'Solo dining']",
       "['Lunch', 'Dinner', 'Solo dining']",
       "['Breakfast', 'Lunch', 'Solo dining']", "['Solo dining']",
       "['Dinner', 'Solo dining']", "['Lunch', 'Solo dining']",
       "['Breakfast', 'Solo dining']", "['Lunch', 'Dinner']",
       "['Breakfast', 'Dinner', 'Solo dining']",
       "['Breakfast', 'Lunch', 'Dinner']", "['Lunch']",
       "['Breakfast', 'Lunch']", "['Breakfast']", "['Dinner']",
       "['Breakfast', 'Solo dining', 'Good for working on laptop']",
       "['Breakfast', 'Lunch', 'Solo dining', 'Good for working on laptop']",
       "['Breakfast', 'Lunch', 'Dinner', 'Solo dining', 'Good for working on laptop']",
       "['Solo dining', 'Good for working on laptop']",
       "['Lunch', 'Dinner', 'Solo dining', 'Good for working on laptop']",
       "['Lunch', 'Solo dining', 'Good for working on laptop']"],
      dtype=object)

In [ ]:
df_expanded_clean['Amenities'].unique()

array([nan, "['Good for kids', 'Restroom']", "['Good for kids']",
       "['Good for kids', 'High chairs', 'Restroom', 'Wi-Fi']",
       "['Gender-neutral restroom', 'Good for kids', 'Restroom']",
       "['Good for kids', 'High chairs']", "['Bar onsite', 'Restroom']",
       "['Good for kids', 'Restroom', 'Wi-Fi']",
       "['Bar onsite', 'Good for kids', 'High chairs']",
       "['Restroom', 'Wi-Fi']", "['Bar onsite']",
       "['Good for kids', 'High chairs', 'Restroom']",
       "['Public restroom', 'Restroom']",
       "['Gender-neutral restroom', 'Good for kids']",
       "['Bar onsite', 'Good for kids']",
       "['Good for kids', 'High chairs', 'Wi-Fi']",
       "['Bar onsite', 'High chairs', 'Restroom', 'Wi-Fi']", "['Wi-Fi']",
       "['Good for kids', 'Wi-Fi']",
       "['Gender-neutral restroom', 'Good for kids', 'High chairs', 'Restroom']",
       "['Bar onsite', 'High chairs']",
       "['Gender-neutral restroom', 'Good for kids', 'High chairs', 'Wi-Fi']",
       "['Bar on

In [ ]:
df_expanded_clean['Atmosphere'].unique()

array([nan, "['Casual', 'Cozy']", "['Casual']",
       "['Casual', 'Cozy', 'Quiet']", "['Cozy']",
       "['Casual', 'Romantic']", "['Casual', 'Historic']",
       "['Casual', 'Trending']", "['Trending']", "['Casual', 'Quiet']",
       "['Cozy', 'Romantic']", "['Quiet', 'Romantic']", "['Romantic']",
       "['Cozy', 'Historic']", "['Casual', 'Cozy', 'Romantic']",
       "['Cozy', 'Romantic', 'Upscale']", "['Upscale']",
       "['Romantic', 'Upscale']", "['Casual', 'Cozy', 'Upscale']",
       "['Casual', 'Cozy', 'Historic']", "['Casual', 'Cosy']",
       "['Cosy', 'Romantic', 'Upmarket']",
       "['Cosy', 'Historic', 'Romantic', 'Upmarket']",
       "['Casual', 'Cozy', 'Trending']",
       "['Casual', 'Cozy', 'Romantic', 'Upscale']",
       "['Casual', 'Cosy', 'Historic']", "['Romantic', 'Upmarket']",
       "['Casual', 'Romantic', 'Trending', 'Upscale']",
       "['Casual', 'Cosy', 'Upmarket']",
       "['Cozy', 'Historic', 'Romantic', 'Upscale']",
       "['Casual', 'Cozy', 'Historic

In [ ]:
df_expanded_clean['Crowd'].unique()

array([nan, "['Groups', 'Tourists']", "['Family-friendly']",
       "['College students', 'Family-friendly']", "['Groups', 'Locals']",
       "['College students', 'Groups']", "['Groups']",
       "['College students', 'Family-friendly', 'Tourists']",
       "['Tourists']", "['Locals']", "['Family-friendly', 'Groups']",
       "['Family-friendly', 'Tourists']", "['College students']",
       "['Family-friendly', 'Groups', 'Tourists']",
       "['Groups', 'Locals', 'Tourists']",
       "['College students', 'Tourists']",
       "['College students', 'Groups', 'Tourists']",
       "['College students', 'Family-friendly', 'Groups']",
       "['College students', 'Locals', 'Tourists']",
       "['College students', 'Groups', 'Locals']",
       "['Locals', 'Tourists']", "['College students', 'Locals']",
       "['Groups', 'Tourists', 'University students']",
       "['Family friendly', 'Tourists', 'University students']",
       "['Family friendly', 'Groups', 'Tourists', 'University student

In [ ]:
columns_to_drop = ['state', 'address', 'hours', 'Service options', 'Accessibility', 'Offerings', 'Highlights', 'From the business', 'Health and safety', 'Health & safety', 'Planning', 'Payments', 'Amenities']
df_processed = df_expanded_clean.drop(columns=columns_to_drop)

In [ ]:
df_processed.isna().sum()

user_id                0
rating                 0
text                   0
gmap_id                0
name_y                 0
latitude               0
longitude              0
category               0
avg_rating             0
num_of_reviews         0
price             228737
Dining options    373764
Popular for       381433
Atmosphere        338579
Crowd             361848
dtype: int64

<div style="background-color:#3F7FBF; color:white; padding:10px"> 

Since our dataset is large enough, for the remaining missing values, we decided to just simple drop the rows with missing `Popular for`, `Dining options`, and `price`.

In [ ]:
df_processed2 = df_processed.dropna(subset=['Popular for', 'Dining options', 'price'])
df_processed2.isna().sum()

user_id               0
rating                0
text                  0
gmap_id               0
name_y                0
latitude              0
longitude             0
category              0
avg_rating            0
num_of_reviews        0
price                 0
Dining options        0
Popular for           0
Atmosphere         1279
Crowd             39439
dtype: int64

<div style="background-color:#3F7FBF; color:white; padding:10px"> 

We can encode the missing values in `Atmosphere` as 'unknown', and the ones in `Crowd` as 'any'.

We will also need to process the representating of `price`. We decided to encode the `$` symbols as numbers, so `$` = 1, `$$` = 2, an so on.


In [ ]:
# Replace NA values in 'Atmosphere' column with 'unknown'
df_processed2.loc[:, 'Atmosphere'] = df_processed2['Atmosphere'].fillna('unknown')

# Replace NA values in 'Crowd' column with 'any'
df_processed2.loc[:, 'Crowd'] = df_processed2['Crowd'].fillna('any')

In [ ]:
df_processed2.isna().sum()

user_id           0
rating            0
text              0
gmap_id           0
name_y            0
latitude          0
longitude         0
category          0
avg_rating        0
num_of_reviews    0
price             0
Dining options    0
Popular for       0
Atmosphere        0
Crowd             0
dtype: int64

In [ ]:
def encode_price(price):
    if pd.isna(price):
        return None  # Use np.nan 
    price = price.replace('₩', '$')  # Normalize '₩' to '$'
    return len(price)  # The number of '$' symbols corresponds to the price level


In [ ]:
df_processed2.loc[:, 'price'] = df_processed2['price'].apply(encode_price)

In [ ]:
df_processed2.head()

,user_id,rating,text,gmap_id,name_y,latitude,longitude,category,avg_rating,num_of_reviews,price,Dining options,Popular for,Atmosphere,Crowd
24,1.085140e+20,5.0,Amazing food and T is an amazing man.,0x89e3169821e62d4d:0x14ff0683c1ebca0e,Three Star Pizza,42.559072,-70.881542,"['Pizza restaurant', 'Italian restaurant', 'De...",3.9,48.0,2,"['Breakfast', 'Lunch', 'Dinner', 'Dessert']","['Breakfast', 'Lunch', 'Dinner', 'Solo dining']","['Casual', 'Cozy']",any
25,1.002992e+20,3.0,I didn't actually try it but just felt like gi...,0x89e3169821e62d4d:0x14ff0683c1ebca0e,Three Star Pizza,42.559072,-70.881542,"['Pizza restaurant', 'Italian restaurant', 'De...",3.9,48.0,2,"['Breakfast', 'Lunch', 'Dinner', 'Dessert']","['Breakfast', 'Lunch', 'Dinner', 'Solo dining']","['Casual', 'Cozy']",any
26,1.176323e+20,5.0,I am a huge fan of their sandwiches! Made to o...,0x89e3169821e62d4d:0x14ff0683c1ebca0e,Three Star Pizza,42.559072,-70.881542,"['Pizza restaurant', 'Italian restaurant', 'De...",3.9,48.0,2,"['Breakfast', 'Lunch', 'Dinner', 'Dessert']","['Breakfast', 'Lunch', 'Dinner', 'Solo dining']","['Casual', 'Cozy']",any
27,1.006278e+20,2.0,There's a reason why it's not called 5 star pi...,0x89e3169821e62d4d:0x14ff0683c1ebca0e,Three Star Pizza,42.559072,-70.881542,"['Pizza restaurant', 'Italian restaurant', 'De...",3.9,48.0,2,"['Breakfast', 'Lunch', 'Dinner', 'Dessert']","['Breakfast', 'Lunch', 'Dinner', 'Solo dining']","['Casual', 'Cozy']",any
28,1.138433e+20,4.0,Drunk and starving at 1am? They're open!,0x89e3169821e62d4d:0x14ff0683c1ebca0e,Three Star Pizza,42.559072,-70.881542,"['Pizza restaurant', 'Italian restaurant', 'De...",3.9,48.0,2,"['Breakfast', 'Lunch', 'Dinner', 'Dessert']","['Breakfast', 'Lunch', 'Dinner', 'Solo dining']","['Casual', 'Cozy']",any


In [ ]:
# df_processed2.to_csv('data/cleaned_data.csv')

In [ ]:
df_final = pd.read_csv('data/cleaned_data.csv')

# 2. EDA

## Exploring the distribution of ratings

<div style="background-color:#3F7FBF; color:white; padding:10px"> 

* Ratings are on a 1 to 5 scale.
* The majority of reviews are positive, with 77870 reviews rated 5.
* However, we note that the second most common rating is 1 at 9435 reviews. This suggests that people leave reviews either when they have a highly good or highly bad experience at a restaurant. 

In [ ]:
# Distribution of ratings: To understand the overall sentiment towards the businesses.

plt.hist(df['rating'], bins = 5, edgecolor = 'black')
plt.title('Distribution of Ratings')
plt.xlabel('Rating')
plt.ylabel('Count')
# show exact counts
for i in range(1, 6):
    plt.text(i + 0.5 if i == 1 else i + 0.3 if i == 2 else i + 0.1 if i == 3 else i if i == 4 else i - 0.2, len(df[df['rating'] == i]), str(len(df[df['rating'] == i])), ha='right', va='bottom')
# change x ticks to read 1, 2, 3, 4, 5
plt.xticks(np.arange(1, 6, 1))

plt.show()


## Exploring review counts per local business

<div style="background-color:#3F7FBF; color:white; padding:10px"> 

* Most businesses have 10-20 reviews, with the number of businesses decreasing at a decreasing rate as the number of reviews increases.
* The mean number of reviews is 20.5 with a standard deviation of 40.53.
* This suggests that a small number of businesses might be overrepresented in our dataset, especially the one outlying businesses with 1292 reviews.

In [ ]:
# Count of reviews per business - To see which businesses have been reviewed the most.

# describe the count of reviews per business
print("Summary statistics for count of reviews per business:")
print(df['gmap_id'].value_counts().describe())

df['gmap_id'].value_counts().plot(kind='hist', bins=100, edgecolor='black')
plt.title('Count of Reviews per Business')
plt.xlabel('Number of Reviews')
plt.ylabel('Count of Businesses')
plt.show()




## Exploring review lengths (in terms of number of characters)

<div style="background-color:#3F7FBF; color:white; padding:10px"> 

* We see that review lengths are distributed similarly to business ratings, with a large number of reviews having under 100 characters, and with the number of reviews decreasing at a decreasing rate as the review length increases.
* The mean review length is 167.47, but with a large standard deviation of 273.39.

In [ ]:
# Review length analysis: To see the distribution of the length of the review texts.

df['review_length'] = df['text'].apply(lambda x: len(x) if x is not None else 0)

# describe the review length
print("Summary statistics for review length:")
print(df['review_length'].describe())

plt.hist(df['review_length'], bins=100, edgecolor='black')
plt.title('Distribution of Review Length')
plt.xlabel('Review Length')
plt.ylabel('Count')
plt.show()



## Exploring correlation between review length and rating

<div style="background-color:#3F7FBF; color:white; padding:10px"> 

* We analyzed the correlation between the length of the review text and the rating. The correlation coefficient is around -0.156, which suggests that there is only a weak negative correlation between the length of the review and the rating.
* This result is consistent with our intuition, as the length of a review does not necessarily indicate its quality or sentiment.
* However, perhaps some angrier customers might leave longer reviews, which could explain the slight negative correlation. 
* We also see that reviews with a rating of 3 have a relatively lower review length, which supports the overlying notion that people write more/longer reviews when they feel strongly about a restaurant. 

In [ ]:
# Correlation between review length and rating: To see if there is a correlation between the length of the review and the rating given.

# plot the correlation between review length and rating
sns.scatterplot(x='rating', y='review_length', data=df)
plt.title('Correlation between Review Length and Rating')
plt.xlabel('Rating')
plt.ylabel('Review Length')
plt.xticks(np.arange(1, 6, 1))
plt.show()

correlation_matrix = df[['rating', 'review_length']].corr()
print("Correlation matrix:")
print(correlation_matrix)

## Number of unique authors

<div style="background-color:#3F7FBF; color:white; padding:10px"> 

* We see that we have 77593 unique authors for our 100,000 reviews, which suggests that most authors have only left one review.
* This is confirmed by the mean number of reviews per author, which is 1.3 with a standard deviation of 0.88.
* In fact, we see through the summary statistics that over 75% of authors have only left 1 review. The most number of reviews left by one author is 62. 
* The shape of the distribution is once again similar to the previous two, with the number of authors decreasing at a decreasing rate as the number of reviews per author increases.

In [ ]:
# How many unique authors?

print("Number of unique authors: ", df['name'].nunique())

# describe the count of ratings per author
print("Summary statistics for number of ratings per author:")
print(df['name'].value_counts().describe())

df['name'].value_counts().plot(kind='hist', bins=100, edgecolor='black')
plt.title('Count of Ratings per Author')
plt.xlabel('Number of Ratings')
plt.ylabel('Count of Authors')
plt.show()


## Simple sentiment analysis - Baseline Model

<div style="background-color:#3F7FBF; color:white; padding:10px"> 

* We used the `TextBlob` library to perform a simple sentiment analysis on the review text.
* First, we dropped reviews with no text, as they would not provide any information for sentiment analysis.
* We then calculated the polarity of each review, which ranges from -1 (most negative) to 1 (most positive).
* Plotting a histogram and summary statistics, we see that most reviews are moderately positive with a mean polarity of 0.36 and a standard deviation of 0.31. This is consistent with our earlier observation that most ratings are positive (5 stars).
* The distribution is roughly bell-shaped, but with a density spike at 0 and a few more spikes above 0.5. 


In [ ]:
# Simple sentiment analysis on review text

# Add a column to the dataframe with the sentiment of the review
df_dropped = df.dropna(subset=['text'])
df_dropped['sentiment'] = df_dropped['text'].apply(lambda x: TextBlob(x).sentiment.polarity)

# print summary statistics
print("Summary statistics for sentiment:")
print(df_dropped['sentiment'].describe())

# plot the distribution of sentiment
plt.hist(df_dropped['sentiment'], bins=100, edgecolor='black')
plt.title('Distribution of Sentiment')
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.show()

<div style="background-color:#3F7FBF; color:white; padding:10px"> 

There is a moderate positive correlation between sentiment score and rating, with a correlation coefficient of 0.547. This suggests that reviews with higher ratings tend to have more positive sentiment scores, which is expected. This is a good result for our training data.

We were worried that meaningful sentiment might not be extracted from short reviews, but the correlation suggests that the sentiment analysis is capturing the sentiment of the reviews decently well. Yay!

In [ ]:
# Find correlation between sentiment and rating
correlation_matrix = df_dropped[['rating', 'sentiment']].corr()
print("Correlation matrix:")
print(correlation_matrix)

## Summary of findings

<div style="background-color:#3F7FBF; color:white; padding:10px"> 

* We see that the majority of ratings given by Google users are 5 out of 5. This aligns with the Google text review sentiment analysis, which found that the majority of the reviews had a postive sentiment.
* We also found that there is a weak correlation between the length of a text review and the rating given by the same Google user. However, it could be helpful to note that based on the visualization displayed above, the review length was longest for either 1 or 5 star reviews.
* In the sentiment analysis, we found that majority of the text reviews result in a sentiment score near 0, which suggests that many reviews are neutral. The second most common sentiment scores are those near 1, which suggests that there are also many (but not as many) reviews that are highly positive. 
* This summarizes the distributional features of our reviews in their length, sentiment and the related score.

## Revised Project Question

<div style="background-color:#3F7FBF; color:white; padding:10px"> 

Creating a restaurant recommendation system based on textual reviews and and user-inputted prompts.


## Implementation Plan -- Draft Pipeline

<div style="background-color:#3F7FBF; color:white; padding:10px"> 

The structure of our problem is similar to that of designing a recommender system. We describe an initial approach that comes to mind:

**Recommender Model Pipeline:**
- **Training step** (preprocessing): Embed the reviews for all restaurants as vectors using sentence embeddings (BERT), or a combined approach using BERT and other feature engineered features from the restaurant information that we have. One approach is then to cluster restaurants in an unsupervised way in the feature space using an approach like KNN or t-SNE. Perhaps we can do this in an informed way, however.
- **Interaction step:** Then, we may take a user prompt of sentence length describing the restaurant type they are looking for, and use the fine-tuned BERT pipeline to get a vector representation of the prompt.
- **Matching step:** Then we can take the top recommendations as the closest three or so restaurants in the feature space to the prompt. We can incorporate additional heuristics in this matching step.
  

**Potential Issues:**
- Reviews and prompts are semantically different. We might encounter unexpected relationships between the restaurants closest in the feature space based on reviews and the inputted prompts.
- Perhaps a content-based filtering approach is more directly applicable as a pipeline (ref: https://developers.google.com/machine-learning/recommendation/content-based/basics).
- With an unsupervised approach like this, we have no systematized way to measure whether our recommending algorithm is doing a good job at matching prompts to restaurants. But this is the nature of trying a recommendation system rather than a predictive model. We do not have the resources to conduct proper A/B testing, which would otherwise be a canonical solution.
-  A collaborative filtering approach would be great, but is infeasible since we cannot recruit enough users. 

# 3. Pipeline and Baseline Model:

#### preprocessing:

Note if you're trying to make this run. Use the cleaned_data 732MB csv file. Change the filepath below. Then it should run all in 15s.

In [5]:
# Load the data
with open('/Users/driesrooryck/Desktop/s24/ac209b/project/restaurant-rec-system/data/cleaned_data.csv', 'r') as f:
    data = f.readlines()

# Convert to dataframe
data_json_str = "[" + ','.join(data) + "]"
df = pd.read_csv(StringIO(data_json_str))

/var/folders/zx/klnklk4d1vq5zns_hyg6xc0h0000gn/T/ipykernel_21844/3734935933.py:7: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(StringIO(data_json_str))


In [6]:
df.dtypes

[                 float64
user_id           float64
rating            float64
text               object
gmap_id            object
name_y             object
latitude          float64
longitude         float64
category           object
avg_rating        float64
num_of_reviews    float64
price             float64
Dining options     object
Popular for        object
Atmosphere         object
Crowd              object
dtype: object

In [11]:
### get 10000 restaurants dumb sample. Take pics out.

# Import the necessary libraries
# Set the random seed for reproducibility
np.random.seed(42)

# Group by 'gmap_id'
grouped = df.groupby('gmap_id')


# If you need to work with all entries for each 'gmap_id' and then select 1000 groups
## Option 1: Select the first 1000 unique restaurants
top_1000_restaurants = grouped.apply(lambda x: x.head(1)).sample(1000)

## Option 2: If you need to select based on the size of each group (e.g., most entries)
# This will sort groups by size and take the top 1000 groups
top_1000_restaurants = grouped.size().nlargest(500).index
selected_entries = df[df['gmap_id'].isin(top_1000_restaurants)]


## Option 3:
# Randomly sample 1000 unique restaurants
# Ensure you have unique restaurants first
unique_restaurants = df.drop_duplicates(subset='gmap_id')
n_0 = 100
if len(unique_restaurants) >= n_0:
    sampled_restaurants = unique_restaurants.sample(n=n_0, random_state=1)  # Use random_state for reproducibility
else:
    print("There are less than 1000 unique restaurants available.")
    sampled_restaurants = unique_restaurants  # Use all available if less than 1000

# Now, get all entries for these 1000 restaurants
selected_entries = df[df['gmap_id'].isin(sampled_restaurants['gmap_id'])]

# Print or inspect the result
display(selected_entries)

# we have 76k rows. This is workable.
df = selected_entries
# Preprocess to remove invalid coordinate restaurants
df = df[(df['latitude'] != 0) | (df['longitude'] != 0)]

/var/folders/zx/klnklk4d1vq5zns_hyg6xc0h0000gn/T/ipykernel_21844/2197141206.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_1000_restaurants = grouped.apply(lambda x: x.head(1)).sample(1000)


,[,user_id,rating,text,gmap_id,name_y,latitude,longitude,category,avg_rating,num_of_reviews,price,Dining options,Popular for,Atmosphere,Crowd
NaN,18090.0,1.159407e+20,5.0,It's the best place to get sushi. The prices a...,0x89e3799ccdf42d39:0x71470e8146b6ca4f,Hoshi Ya Sushi,42.349175,-71.162908,['Japanese restaurant'],4.4,26.0,2.0,"['Lunch', 'Dinner']","['Lunch', 'Dinner', 'Solo dining']",['Casual'],"['Family-friendly', 'Groups']"
NaN,18091.0,1.154406e+20,5.0,Wonderful place! By far the best sweet potato ...,0x89e3799ccdf42d39:0x71470e8146b6ca4f,Hoshi Ya Sushi,42.349175,-71.162908,['Japanese restaurant'],4.4,26.0,2.0,"['Lunch', 'Dinner']","['Lunch', 'Dinner', 'Solo dining']",['Casual'],"['Family-friendly', 'Groups']"
NaN,18092.0,1.123766e+20,5.0,I call this excellent place often because I li...,0x89e3799ccdf42d39:0x71470e8146b6ca4f,Hoshi Ya Sushi,42.349175,-71.162908,['Japanese restaurant'],4.4,26.0,2.0,"['Lunch', 'Dinner']","['Lunch', 'Dinner', 'Solo dining']",['Casual'],"['Family-friendly', 'Groups']"
NaN,18093.0,1.166474e+20,1.0,The woman who works there is extremely rude. T...,0x89e3799ccdf42d39:0x71470e8146b6ca4f,Hoshi Ya Sushi,42.349175,-71.162908,['Japanese restaurant'],4.4,26.0,2.0,"['Lunch', 'Dinner']","['Lunch', 'Dinner', 'Solo dining']",['Casual'],"['Family-friendly', 'Groups']"
NaN,18094.0,1.135679e+20,5.0,No complaints on their sushi! The tempura crun...,0x89e3799ccdf42d39:0x71470e8146b6ca4f,Hoshi Ya Sushi,42.349175,-71.162908,['Japanese restaurant'],4.4,26.0,2.0,"['Lunch', 'Dinner']","['Lunch', 'Dinner', 'Solo dining']",['Casual'],"['Family-friendly', 'Groups']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,3761246.0,1.069684e+20,5.0,"(Translated by Google) Amazing place\n,\n,(Ori...",0x89e37a04d95ef3ef:0xd27a44c0168836aa,Uni,42.348676,-71.088800,"['Sushi restaurant', 'Asian restaurant', 'Bar'...",4.3,455.0,4.0,"['Dinner', 'Dessert', 'Seating']","['Lunch', 'Dinner', 'Solo dining']",['Casual'],"['College students', 'Family-friendly', 'Group..."
NaN,3761247.0,1.165813e+20,5.0,(Translated by Google) Absolutely brilliant an...,0x89e37a04d95ef3ef:0xd27a44c0168836aa,Uni,42.348676,-71.088800,"['Sushi restaurant', 'Asian restaurant', 'Bar'...",4.3,455.0,4.0,"['Dinner', 'Dessert', 'Seating']","['Lunch', 'Dinner', 'Solo dining']",['Casual'],"['College students', 'Family-friendly', 'Group..."
NaN,3761248.0,1.006766e+20,5.0,(Translated by Google) Japanese fusion food fr...,0x89e37a04d95ef3ef:0xd27a44c0168836aa,Uni,42.348676,-71.088800,"['Sushi restaurant', 'Asian restaurant', 'Bar'...",4.3,455.0,4.0,"['Dinner', 'Dessert', 'Seating']","['Lunch', 'Dinner', 'Solo dining']",['Casual'],"['College students', 'Family-friendly', 'Group..."
NaN,3761249.0,1.164222e+20,2.0,"(Translated by Google) It's too unpalatable, a...",0x89e37a04d95ef3ef:0xd27a44c0168836aa,Uni,42.348676,-71.088800,"['Sushi restaurant', 'Asian restaurant', 'Bar'...",4.3,455.0,4.0,"['Dinner', 'Dessert', 'Seating']","['Lunch', 'Dinner', 'Solo dining']",['Casual'],"['College students', 'Family-friendly', 'Group..."


### Stage 1: filter by basic critiria

In [8]:
!pip install geocoder
!pip install geopy

In [12]:
import geocoder
import pandas as pd
from geopy.distance import geodesic

# Get user coordinates
def get_user_coordinates():
    g = geocoder.ip('me')
    if g.latlng:
        return g.latlng
    else:
        return None

coordinates = get_user_coordinates()
if coordinates:
    user_latitude, user_longitude = coordinates
    print(f"User's coordinates: Latitude={user_latitude}, Longitude={user_longitude}")
else:
    print("Unable to retrieve user's coordinates.")
    # Define default values or handle lack of coordinates appropriately
    # user_latitude, user_longitude = default_latitude, default_longitude


# Function to calculate distance using geopy
def calculate_distance(row, user_lat, user_lon):
    user_location = (user_lat, user_lon)
    business_location = (row['latitude'], row['longitude'])
    return geodesic(user_location, business_location).miles

# Apply the distance function
df['distance'] = df.apply(lambda row: calculate_distance(row, user_latitude, user_longitude), axis=1)
df_filtered = df[df['distance'] <= 10]  # Filter data within 10 miles

# Print the filtered DataFrame
print(df_filtered)


User's coordinates: Latitude=42.3751, Longitude=-71.1056
             [       user_id  rating  \
NaN    18090.0  1.159407e+20     5.0   
NaN    18091.0  1.154406e+20     5.0   
NaN    18092.0  1.123766e+20     5.0   
NaN    18093.0  1.166474e+20     1.0   
NaN    18094.0  1.135679e+20     5.0   
..         ...           ...     ...   
NaN  3761246.0  1.069684e+20     5.0   
NaN  3761247.0  1.165813e+20     5.0   
NaN  3761248.0  1.006766e+20     5.0   
NaN  3761249.0  1.164222e+20     2.0   
NaN  3761250.0  1.168364e+20     5.0   

                                                  text  \
NaN  It's the best place to get sushi. The prices a...   
NaN  Wonderful place! By far the best sweet potato ...   
NaN  I call this excellent place often because I li...   
NaN  The woman who works there is extremely rude. T...   
NaN  No complaints on their sushi! The tempura crun...   
..                                                 ...   
NaN  (Translated by Google) Amazing place\n,\n,(Ori...  

Only 5k reviews left!

Issue to be adressed in final model: for 1.6M reviews, this search-method is intractable.

In [ ]:
### filter by opening hour -- DHATI

# get current time from user

# ! must preprocess opening time well

#  filter dataset by what is still open

In [ ]:
### lookup dish: DHATI, JANICE

# get dish from user as a prompt

# go restaurant by restaurant, then dish by dish, and look all reviews that mention prompt (my best guess is use regex, but think intelligently. pasta alfredo vs alfredo pasta)

### Stage 2: rank by sentiment for the dish - SUSANNAH, ETHAN.

In [ ]:
### get sentiment

# per restaurant, go thru all relevant reviews and take median of sentiment. (maybe smarter way, Susannah?)

# per restaurant, get avg_rating, other metrics we want (like price?). Make weighted index = 0.8 * sentiment + 0.2 * avg_rating.

# return top 3 restaurants by weighted index.